In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import os

# Define the path to your folder
train_dir = '/content/drive/My Drive/Third Year/Spring Quarter/POLI 179/UCLA-protest/img/train'
test_dir = '/content/drive/My Drive/Third Year/Spring Quarter/POLI 179/UCLA-protest/img/test'

# Check if the paths exist
print("Train directory exists:", os.path.exists(train_dir))
print("Test directory exists:", os.path.exists(test_dir))

# List the contents of the directories
try:
    print("Contents of train directory:", os.listdir(train_dir))
    print("Contents of test directory:", os.listdir(test_dir))
except OSError as e:
    print(f"Error accessing directory: {e}")


Train directory exists: True
Test directory exists: True
Contents of train directory: ['train-13934.jpg', 'train-00808.jpg', 'train-23353.jpg', 'train-30018.jpg', 'train-21164.jpg', 'train-12567.jpg', 'train-18584.jpg', 'train-11010.jpg', 'train-04250.jpg', 'train-27050.jpg', 'train-14001.jpg', 'train-11829.jpg', 'train-26152.jpg', 'train-00274.jpg', 'train-26561.jpg', 'train-01211.jpg', 'train-14733.jpg', 'train-28302.jpg', 'train-00282.jpg', 'train-19802.jpg', 'train-26631.jpg', 'train-03203.jpg', 'train-21348.jpg', 'train-15236.jpg', 'train-03813.jpg', 'train-02973.jpg', 'train-28123.jpg', 'train-05441.jpg', 'train-20897.jpg', 'train-17546.jpg', 'train-06046.jpg', 'train-00022.jpg', 'train-02950.jpg', 'train-31601.jpg', 'train-31407.jpg', 'train-14524.jpg', 'train-09084.jpg', 'train-25491.jpg', 'train-19582.jpg', 'train-20934.jpg', 'train-15975.jpg', 'train-17739.jpg', 'train-23538.jpg', 'train-05638.jpg', 'train-12459.jpg', 'train-13580.jpg', 'train-00127.jpg', 'train-29724.jpg', '

In [ ]:
!cp -r '/content/drive/My Drive/Third Year/Spring Quarter/POLI 179/UCLA-protest/img/train' /content/train
!cp -r '/content/drive/My Drive/Third Year/Spring Quarter/POLI 179/UCLA-protest/img/test' /content/test

# Update paths to the local filesystem
train_dir = '/content/train'
test_dir = '/content/test'

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
def load_and_preprocess_image(path, target_size=(150, 150)):
    img = load_img(path, target_size=target_size)
    img = img_to_array(img)
    img = img / 255.0  # Normalize to [0, 1]
    return img

def load_images_from_directory(directory, start_label=0):
    images = []
    labels = []
    for filename in os.listdir(directory):
        if filename.endswith('.jpg'):
            img_path = os.path.join(directory, filename)
            img = load_and_preprocess_image(img_path)
            images.append(img)

            # Assign label based on filename or other criteria
            if 'train' in filename:
                labels.append(start_label)  # Modify this to assign appropriate labels
            elif 'test' in filename:
                labels.append(start_label + 1)  # Modify this to assign appropriate labels
    return np.array(images), np.array(labels)

In [ ]:
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(1000).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(32)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG16

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))  # Modify output layer based on your labels

base_model.trainable = False  # Freeze the base model

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10
)